In [2]:
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
# Scikit-learn imports
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Qiskit impor
from qiskit.primitives import Sampler, Estimator

# Assuming 'backend' is already defined
# For sampling:
from qiskit_machine_learning.algorithms import QSVC
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import Estimator as QiskitEstimator

from qiskit_machine_learning.kernels import TrainableFidelityQuantumKernel
from qiskit_machine_learning.kernels.algorithms import QuantumKernelTrainer
from qiskit_machine_learning.utils.loss_functions import SVCLoss


In [3]:
'''try:
    import qiskit_ibm_runtime
    print("Successfully imported!")
    print(qiskit_ibm_runtime.__version__)
except ModuleNotFoundError as e:
    print(f"Import failed: {e}")
    import sys
    print("Python path:", sys.path)
from qiskit_ibm_runtime import QiskitRuntimeService


QiskitRuntimeService.save_account(channel="ibm_quantum", token="b11763c075e1456a7c3fadca5817c1d82f8fdb5a13cab3d7de7a58c2c437b6efee63880b64d2e5c08c893154ce5908b670a8d2ac4eaeec12f3df55180a5f8701", set_as_default=True, overwrite=True)
 
# Load saved credentials
service = QiskitRuntimeService()
service.backends()'''

Successfully imported!
0.33.2


RequestsApiError: '403 Client Error: Forbidden for url: https://auth.quantum-computing.ibm.com/api/users/loginWithToken. Your IBM Quantum account has been disabled. Learn more: https://ibm.biz/BdfaME., Error code: 3485.'

In [3]:
import os

# Specify the new directory path
new_directory = '/Users/simonwang/Desktop/PY536'
# Change to the new directory
os.chdir(new_directory)
# Print the current working directory to confirm the change
print("Files in Current Directory:", os.listdir('.'))

Files in Current Directory: ['PY536-2024F-HW1.pdf', '.DS_Store', '12_Support_Vector_Machines.pdf', 'QSVM1.ipynb', '_Quantum_Machine_Learning_for_Image_Classification_.pdf', 'Quantum Support Vector Machines.ipynb', 'path', 'Swarm_Behaviour.csv', '.ipynb_checkpoints', 'heart_2020_cleaned.csv', 'quantum-computation-and-quantum-information-nielsen-chuang.pdf', 'qsvm.py', 'QSVM_local.ipynb']


In [5]:

# Load the new dataset
data = pd.read_csv('heart_2020_cleaned.csv')
data= data.sample(n=100, random_state=42)
# Inspect the first few rows to understand the data structure
print(data.head())

# Inspect data types and check for missing values
print(data.info())

# Preprocessing: Convert categorical variables to numeric
# We'll use LabelEncoder for simplicity here; you might choose one-hot encoding or other methods depending on the specific needs
label_encoders = {}
for column in data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column].astype(str))
    label_encoders[column] = le

# Assuming the last column is the target variable
target = 'HeartDisease'  # You should set this according to your dataset's specific target variable

# Split the dataset into features and the target variable
X = data.drop(target, axis=1)
y = data[target]

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Now X_train, X_test, y_train, and y_test are ready to be used for model training
print(f"Training set shape: {X_train.shape}, Testing set shape: {X_test.shape}")

       HeartDisease    BMI Smoking AlcoholDrinking Stroke  PhysicalHealth  \
271884           No  27.63     Yes              No     No             0.0   
270361           No  21.95      No              No     No             0.0   
219060           No  31.32     Yes              No     No             0.0   
24010            No  40.35      No              No     No            30.0   
181930           No  35.61     Yes              No     No            30.0   

        MentalHealth DiffWalking     Sex AgeCategory      Race Diabetic  \
271884          25.0          No  Female       25-29  Hispanic       No   
270361          20.0          No  Female       30-34     White       No   
219060           0.0          No  Female       40-44     White       No   
24010            0.0          No  Female       65-69     White       No   
181930          30.0         Yes  Female       60-64     White       No   

       PhysicalActivity  GenHealth  SleepTime Asthma KidneyDisease SkinCancer  
271884

In [6]:
def classical_svm(X_train, X_test, y_train, y_test, kernel='rbf', scale_data=False, verbose=False):
    """
    Train and evaluate a classical Support Vector Machine with optional data scaling.
    
    Args:
        X_train, X_test (array-like): Training and testing feature sets.
        y_train, y_test (array-like): Training and testing labels.
        kernel (str): Kernel type for the SVM. Default is 'rbf'.
        scale_data (bool): If True, scale features before training. Default is False.
        verbose (bool): If True, print additional details during processing. Default is False.
    
    Returns:
        Dictionary with performance metrics and training time.
    """
    start_time = time.time()
    
    # Optionally scale the data
    if scale_data:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        if verbose:
            print("Data scaling applied.")
    
    # Create and train SVM
    svm_classifier = SVC(kernel=kernel)
    svm_classifier.fit(X_train, y_train)
    
    if verbose:
        print("SVM training completed.")
    
    # Predict and evaluate
    y_pred = svm_classifier.predict(X_test)
    end_time = time.time()
    
    # Prepare the performance metrics
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'training_time': end_time - start_time,
        'classification_report': classification_report(y_test, y_pred, output_dict=True)
    }
    
    if verbose:
        print(f"Classification Report:\n {classification_report(y_test, y_pred)}")
    
    return metrics



In [26]:
from qiskit import transpile
import time
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.primitives import Sampler
from qiskit_ibm_runtime import QiskitRuntimeService
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(message)s")
def compute_kernel_matrix(sampler, feature_map, x_vec, y_vec=None):
    """
    Compute the kernel matrix using the Sampler.
    """
    if y_vec is None:
        y_vec = x_vec

    # Create circuits for all combinations of x_vec and y_vec
    circuits = []
    for x in x_vec:
        for y in y_vec:
            qc_x = feature_map.assign_parameters(x)  # Assign x parameters
            qc_y = feature_map.assign_parameters(y)  # Assign y parameters
            qc = qc_x.compose(qc_y.inverse())  # Combine x and y circuits
            circuits.append(qc)

    # Transpile and execute circuits
    transpiled_circuits = transpile(circuits, backend=sampler)
    results = sampler.run(transpiled_circuits).result()

    # Extract fidelities
    fidelities = np.array(results.fidelities)
    return fidelities.reshape(len(x_vec), len(y_vec))



In [27]:
def quantum_svm_with_backend(X_train, X_test, y_train, y_test, backend_name='ibm_brisbane'):
    """
    Train and evaluate Quantum Support Vector Machine using a real IBM Quantum backend.
    
    Args:
        X_train, X_test: Training and testing feature sets.
        y_train, y_test: Training and testing labels.
        backend_name: Name of the IBM Quantum backend to use.
    
    Returns:
        Dictionary with performance metrics and training time.
    """
    logging.info("Initializing IBM Quantum service.")
    service = QiskitRuntimeService()

    # Select the backend
    logging.info(f"Selecting backend: {backend_name}")
    backend = service.backend(backend_name)

    # Ensure the feature dimension matches the dataset dimensions
    feature_dimension = X_train.shape[1]
    logging.info(f"Feature dimension (number of qubits): {feature_dimension}")

    # Scale data
    logging.info("Scaling data to [0, 1] range.")
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Create the feature map
    logging.info("Creating ZZFeatureMap.")
    feature_map = ZZFeatureMap(feature_dimension, reps=3, entanglement='full')

    # Initialize Sampler
    sampler = Sampler()

    # Compute kernel matrices
    logging.info("Computing kernel matrix for training.")
    start_time = time.time()
    kernel_train = compute_kernel_matrix(sampler, feature_map, X_train_scaled)

    logging.info("Computing kernel matrix for testing.")
    kernel_test = compute_kernel_matrix(sampler, feature_map, X_test_scaled, X_train_scaled)

    # Train classical SVM using the precomputed kernel matrix
    logging.info("Training classical SVM with quantum kernel.")
    svm = SVC(kernel="precomputed")
    svm.fit(kernel_train, y_train)

    # Predict
    logging.info("Making predictions.")
    y_pred = svm.predict(kernel_test)

    # End timing
    end_time = time.time()
    training_time = end_time - start_time
    logging.info(f"Training and testing completed in {training_time:.4f} seconds.")

    # Evaluate results
    return {
        'accuracy': accuracy_score(y_test, y_pred),
        'training_time': training_time,
        'classification_report': classification_report(y_test, y_pred)
    }
   

In [28]:


# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
# Train and evaluate classical SVM
classical_results = classical_svm(X_train, X_test, y_train, y_test)
print("Classical SVM Results:")
print(f"Accuracy: {classical_results['accuracy']:.4f}")
print(f"Training Time: {classical_results['training_time']:.4f} seconds")
print(classical_results['classification_report'])


Classical SVM Results:
Accuracy: 0.8500
Training Time: 0.0050 seconds
{'0': {'precision': 0.85, 'recall': 1.0, 'f1-score': 0.918918918918919, 'support': 17.0}, '1': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 3.0}, 'accuracy': 0.85, 'macro avg': {'precision': 0.425, 'recall': 0.5, 'f1-score': 0.4594594594594595, 'support': 20.0}, 'weighted avg': {'precision': 0.7224999999999999, 'recall': 0.85, 'f1-score': 0.7810810810810811, 'support': 20.0}}


/Users/simonwang/path/to/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/simonwang/path/to/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/simonwang/path/to/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r

In [10]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Initialize the IBM Quantum service
service = QiskitRuntimeService()

# List available backends
print(service.backends())


qiskit_runtime_service.__init__:INFO:2024-12-03 11:57:13,563: Default instance: ibm-q/open/main


[<IBMBackend('ibm_brisbane')>, <IBMBackend('ibm_kyiv')>, <IBMBackend('ibm_sherbrooke')>]


In [11]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Initialize the IBM Quantum service
service = QiskitRuntimeService()

# List available backends
backends = service.backends()
print("Available backends:", [backend.name for backend in backends])

# Select a backend with sufficient qubits
backend = next(backend for backend in backends if backend.configuration().n_qubits >= 5 and not backend.configuration().simulator)
print(f"Selected backend: {backend.name}")

qiskit_runtime_service.__init__:INFO:2024-12-03 11:57:21,871: Default instance: ibm-q/open/main


Available backends: ['ibm_brisbane', 'ibm_kyiv', 'ibm_sherbrooke']
Selected backend: ibm_brisbane


In [29]:

results = quantum_svm_with_backend(X_train, X_test, y_train, y_test, backend_name='ibm_brisbane')

print("Quantum SVM Results:")
print(f"Accuracy: {results['accuracy']:.4f}")
print(f"Training Time: {results['training_time']:.4f} seconds")
print("Classification Report:")
print(results['classification_report'])

2024-12-03 12:15:24,977 - Initializing IBM Quantum service.
qiskit_runtime_service.__init__:INFO:2024-12-03 12:15:29,273: Default instance: ibm-q/open/main
2024-12-03 12:15:29,274 - Selecting backend: ibm_brisbane
2024-12-03 12:15:31,401 - Feature dimension (number of qubits): 17
2024-12-03 12:15:31,402 - Scaling data to [0, 1] range.
2024-12-03 12:15:31,403 - Creating ZZFeatureMap.
/var/folders/8n/5y209hpd0dq56l8hhhtwg25c0000gn/T/ipykernel_11093/3434977756.py:35: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()
2024-12-03 12:15:31,403 - Computing kernel matrix for training.


AttributeError: 'Sampler' object has no attribute 'configuration'